In [ ]:
# TẢI DỮ LIỆU VÀ CÁC PACKAGE CẦN THIẾT  

import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns 
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

file_path = "../data/final_data.xlsx"
xls = pd.ExcelFile(file_path)
input_df = xls.parse(sheet_name="Sheet2")

# display(input_df.head(5))
# display(input_df.shape) # có 341 hàng và 84 cột
# display(input_df.columns) # có 84 cột, một số tên không thuận tiện cho thao tác, và một số cột sẽ không cần thiết 

In [21]:
# TẠO HAI CỘT HOMA_IR VÀ TYG ĐỂ ĐÁNH GIÁ ĐỀ KHÁNG INSULIN

input_df["homa_ir"] = (input_df["insulin"] * input_df["glucose_ac"]) / 405

# original_df["TG"].isna().value_counts() # có 33 đối tượng thiếu TG
input_df["tyg"] = np.log((input_df["TG"] * input_df["glucose_ac"]) / 2)

In [22]:
# BÁO CÁO % GIÁ TRỊ BỊ THIẾU Ở CÁC LOẠI BIẾN SỐ

high_missing_columns = []

def report_missing_percentage(col):
    missing_percent = ((input_df[col].isna().mean() * 100).round(2))
    if missing_percent > 40:
        high_missing_columns.append(col)
    # print(f"The {col} column has a missing data rate of {missing_percent}%.")

for col in input_df.columns:
    report_missing_percentage(col)

# display(high_missing_columns)
# display(len(high_missing_columns)) # có 25 cột thiếu > 40% số giá trị


In [23]:
# LOẠI BỎ CÁC CỘT CÓ SỐ LƯỢNG THIẾU > 40%

df_1 = input_df.drop(columns=high_missing_columns, inplace=False).copy()

df_1.shape # (341, 61) --- 86 - 25 = 61

(341, 61)

In [24]:
# LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT 

removed_cols = ["number", "hospital", 'blood_date', 'date_follow', 'body_date', 'birth']

df_1.drop(columns=removed_cols, inplace=True)

# df_1.shape # (341, 55) --- 61 - 6 = 55


In [25]:
# TẠO CODE BOOK CHO CÁC BIẾN SỐ ĐỊNH TÍNH

cat_cols = ['sex',
 'pregnancy_smoking',
 'GDM',
 'preterm_birth',
 'father_diabetes',
 'mother_diabetes',
 'education_level']

# thiếu cột CPP rồi

sex_code = {"female": 0, "male": 1}
pregnancy_smoking_code = {"no": 0, "yes": 1}
GDM_code = {"no-dia": 1, "gmd": 2, "other-dia": 3}
preterm_birth_code = {"no": 0, "yes": 1}
father_diabetes_code = {"no": 0, "yes": 1}
mother_diabetes_code = {"no": 0, "yes": 1}
education_level_code = {"senior high": 1, "vocational": 2, "graduate": 3, "college": 4}


In [26]:
# MÃ HÓA CÁC BIẾN ĐỊNH TÍNH

df_1["sex"] = df_1["sex"].map(sex_code).astype("category")
df_1["pregnancy_smoking"] = df_1["pregnancy_smoking"].map(pregnancy_smoking_code).astype("category")
df_1["GDM"] = df_1["GDM"].map(GDM_code).astype("category")
df_1["preterm_birth"] = df_1["preterm_birth"].map(preterm_birth_code).astype("category")
df_1["father_diabetes"] = df_1["father_diabetes"].map(father_diabetes_code).astype("category")
df_1["mother_diabetes"] = df_1["mother_diabetes"].map(mother_diabetes_code).astype("category")
df_1["education_level"] = df_1["education_level"].map(education_level_code).astype("category")
df_1["CPP"] = df_1["CPP"].astype("category")

# nên chuyển thành category mà không phải int vì giữ dạng category để mice package có thể chọn phương thức impute phù hợp

In [27]:
# CHỌN RA CÁC CỘT THEO ĐỊNH DẠNG DỮ LIỆU

numeric_cols = df_1.select_dtypes(["int64", "float64"]).columns.to_list()
cat_cols = df_1.select_dtypes(["category"]).columns.to_list()

# display(cat_cols) 
display(len(cat_cols))
# display(numeric_cols)
display(len(numeric_cols)) 

8

47

In [28]:
df_1.columns

Index(['age', 'zbmi', 'bmi', 'insulin', 'glucose_ac', 'cholesterol', 'TG',
       'HDL', 'cortisol', 'waist', 'hip', 'SBP', 'DBP', 'waist_hip_ratio',
       'height', 'weight', 'CPP', 'sex', 'family_income', 'pregnancy_smoking',
       'GDM', 'gestational_weight_gain', 'preterm_birth',
       'gestational_age_week', 'birth_weight_gram',
       'exclusive_breastfeeding_month', 'mixed_breastfeeding_month',
       'father_diabetes', 'mother_diabetes', 'f_height', 'f_weight',
       'm_height', 'm_weight', 'education_level', 'father_BMI', 'mother_BMI',
       'sedentary_lifestyle_hour_day', 'low_physical_activity_hour_day',
       'sleep_duration', 'PSQI_score', 'snoring_times_week',
       'sleep_apnea_times_week', 'food_weight(g)', 'calories(kcal)', 'fat(g)',
       'protein(g)', 'total_carbohydrates(g)', 'score_5A', 'PSS', 'score_5B',
       'CES_D', 'score_5C', 'RSE', 'homa_ir', 'tyg'],
      dtype='object')

In [29]:
removed_cols = [
    "bmi", 
    "waist_hip_ratio", 
    "height", 
    "weight", 
    "f_height", 
    "f_weight", 
    "m_height", 
    "m_weight", 
    "sleep_duration", 
    "PSQI_score", 
    "snoring_times_week", 
    "sleep_apnea_times_week", 
    "homa_ir", "tyg", 
    "PSS", "CES_D", "RSE"
]

display(len(removed_cols))

risk_factors = df_1.drop(columns=removed_cols, inplace=False).copy()

display(risk_factors.shape)


17

(341, 38)

In [ ]:
# XUẤT THÀNH BẢNG DỮ LIỆU

# risk_factors.to_excel("risk_factors.xlsx", index=False)

In [31]:
# ĐÁNH GIÁ MỨC ĐỘ ĐA CỘNG TUYẾN 

# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import pandas as pd

# X = df_1.select_dtypes(include=[float, int]).dropna()
# vif_data = pd.DataFrame()
# vif_data["Variable"] = X.columns
# vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# print(vif_data)

# df_1.corr().abs().round(2)
# df_1.nunique()